In [1]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
import requests
import json
import os
import re
import time
from tqdm import tqdm
from glob import glob
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
cookies = {
    'twk_uuid_60b5a146de99a4282a1aa730': '%7B%22uuid%22%3A%221.PUklHkPqHxtTCxog0uFm7cTNNiK4MFsGBwPt5f3jOitFjNR1FGW9u4oBywFGgIr843vL8mdXeQ1jurSjUYCc2Ebu02QB3buhB4deBhi1fj1S8senc%22%2C%22version%22%3A3%2C%22domain%22%3A%22um.edu.my%22%2C%22ts%22%3A1679020786888%7D',
    '_ga_9F5XS8KEPF': 'GS1.1.1679020774.2.1.1679021374.0.0.0',
    '_ga_EQVDTZTY1B': 'GS1.1.1680497726.1.0.1680497742.0.0.0',
    '_ga': 'GA1.3.1274739283.1679018039',
    '__utmz': '168646693.1684772685.1.1.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided)',
    '__utma': '168646693.1274739283.1679018039.1684772685.1684917419.2',
    '__utmc': '168646693',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en-MY,en;q=0.9,en-US;q=0.8,ms;q=0.7',
    'Cache-Control': 'no-cache',
    'Connection': 'keep-alive',
    # 'Cookie': 'twk_uuid_60b5a146de99a4282a1aa730=%7B%22uuid%22%3A%221.PUklHkPqHxtTCxog0uFm7cTNNiK4MFsGBwPt5f3jOitFjNR1FGW9u4oBywFGgIr843vL8mdXeQ1jurSjUYCc2Ebu02QB3buhB4deBhi1fj1S8senc%22%2C%22version%22%3A3%2C%22domain%22%3A%22um.edu.my%22%2C%22ts%22%3A1679020786888%7D; _ga_9F5XS8KEPF=GS1.1.1679020774.2.1.1679021374.0.0.0; _ga_EQVDTZTY1B=GS1.1.1680497726.1.0.1680497742.0.0.0; _ga=GA1.3.1274739283.1679018039; __utmz=168646693.1684772685.1.1.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); __utma=168646693.1274739283.1679018039.1684772685.1684917419.2; __utmc=168646693',
    'Pragma': 'no-cache',
    'Referer': 'http://100.93.25.29:8888/',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'cross-site',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}

In [4]:
def download(url, file_name, retry = 2):
    if os.path.exists(file_name) and os.path.getsize(file_name) > 50000:
        return
    with open(file_name, "wb") as file:
        for k in range(retry):
            try:
                response = requests.get(url, verify = False, timeout = 180, cookies=cookies,
    headers=headers,)
                file.write(response.content)
                return 
            except Exception as e:
                print(k, e, url)
                time.sleep(0.5)

In [5]:
files = glob('pages/*.json')
len(files)

18787

In [6]:
pdfs = []
for f in files:
    with open(f) as fopen:
        pdfs.extend(json.load(fopen))
        
pdfs = list(set(pdfs))

In [7]:
filtered_pdfs = []
for p in tqdm(pdfs):
    if len(re.findall( r'[0-9]+(?:\.[0-9]+){3}', p)):
        continue
    if not p.endswith('.pdf'):
        continue
    if 'ade.sagepub.com' in p:
        continue
    filtered_pdfs.append(p)
    
filtered_pdfs = sorted(list(set(filtered_pdfs)))
len(filtered_pdfs)

100%|███████████████████████████████| 235433/235433 [00:00<00:00, 652923.92it/s]


235130

In [8]:
um = [f for f in filtered_pdfs if 'um.edu.my' in f]
len(um)

52433

In [9]:
# !rm -rf pdf
!mkdir um

mkdir: cannot create directory ‘um’: File exists


In [ ]:
max_worker = 1

um = sorted(list(um))
for i in tqdm(range(0, len(um), max_worker)):
    urls = [(url, os.path.join('um', f'{i}-{no}.pdf')) for no, url in enumerate(um[i: i + max_worker])]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(download, url[0], url[1]): url for url in urls}

        for future in as_completed(futures):
            future.result()

  0%|                                       | 5/52433 [00:01<3:19:39,  4.38it/s]

0 HTTPConnectionPool(host='dspace.fsktm.um.edu.my', port=80): Max retries exceeded with url: /bitstream/1812/220/1/TEL_AA.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f0a7a0a40a0>: Failed to establish a new connection: [Errno -2] Name or service not known')) http://dspace.fsktm.um.edu.my/bitstream/1812/220/1/TEL_AA.pdf
1 HTTPConnectionPool(host='dspace.fsktm.um.edu.my', port=80): Max retries exceeded with url: /bitstream/1812/220/1/TEL_AA.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f0a7a0a4640>: Failed to establish a new connection: [Errno -2] Name or service not known')) http://dspace.fsktm.um.edu.my/bitstream/1812/220/1/TEL_AA.pdf


  0%|                                       | 6/52433 [00:02<8:05:13,  1.80it/s]

0 HTTPConnectionPool(host='e-journal.um.edu.my', port=80): Max retries exceeded with url: /filebank/published_article/10291/JDBE%20Vol%2016%20Issue%202016.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f0a7a0a41f0>: Failed to establish a new connection: [Errno -2] Name or service not known')) http://e-journal.um.edu.my/filebank/published_article/10291/JDBE Vol 16 Issue 2016.pdf
1 HTTPConnectionPool(host='e-journal.um.edu.my', port=80): Max retries exceeded with url: /filebank/published_article/10291/JDBE%20Vol%2016%20Issue%202016.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f0a7a0a4310>: Failed to establish a new connection: [Errno -2] Name or service not known')) http://e-journal.um.edu.my/filebank/published_article/10291/JDBE Vol 16 Issue 2016.pdf


  0%|                                      | 7/52433 [00:03<10:32:52,  1.38it/s]

...

0 HTTPConnectionPool(host='ejum.fsktm.um.edu.my', port=80): Read timed out. (read timeout=180) http://ejum.fsktm.um.edu.my/article/1059.pdf
1 HTTPConnectionPool(host='ejum.fsktm.um.edu.my', port=80): Read timed out. (read timeout=180) http://ejum.fsktm.um.edu.my/article/1059.pdf


  0%|                                  | 22/52433 [24:20<4003:44:24, 275.01s/it]

0 HTTPConnectionPool(host='ejum.fsktm.um.edu.my', port=80): Read timed out. (read timeout=180) http://ejum.fsktm.um.edu.my/article/1062.pdf
1 HTTPConnectionPool(host='ejum.fsktm.um.edu.my', port=80): Read timed out. (read timeout=180) http://ejum.fsktm.um.edu.my/article/1062.pdf


  0%|                                  | 23/52433 [30:22<4381:14:07, 300.94s/it]

0 HTTPConnectionPool(host='ejum.fsktm.um.edu.my', port=80): Read timed out. (read timeout=180) http://ejum.fsktm.um.edu.my/article/1097.pdf


In [11]:
!du -hs um

20G	um
